In [2]:
require('ranger')
require('party')

Loading required package: party
Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich


In [21]:
X.train.dgi <- read.csv("../intermediate/02_enhancer/permuted1_X_train.csv", header = FALSE)
y.train.dgi <- read.csv("../intermediate/02_enhancer/permuted1_y_train.csv", header = FALSE)
noisy_features <- read.csv('../intermediate/02_enhancer/permuted1_noisy_features.csv', header = FALSE)

set.seed(2356789)
data.dgi <- data.frame(X.train.dgi, y = y.train.dgi[, 1])
dgi.rf = ranger(y ~ ., data = data.dgi, 
       num.trees = 100,
       mtry = 3,
       importance = "impurity",
       min.node.size = 20)
imp_org = dgi.rf$variable.importance
imp_org = imp_org/sum(imp_org)

dgi.perm = ranger(y ~ ., data = data.dgi, 
                num.trees = 100,
                mtry = 3,
                importance = "impurity_corrected",
                 min.node.size = 20)
imp_ranger = dgi.perm$variable.importance
imp_ranger = imp_ranger/sum(imp_ranger)

dgi.cforest = cforest(y~., data = data.dgi,
                           control = cforest_control(ntree = 100, 
                                                     mtry = 3, 
                                                     maxdepth = 20))
imp_cforest = varimp(dgi.cforest)
imp_cforest = imp_cforest/sum(imp_cforest)

imp = data.frame(imp_org, imp_ranger, imp_cforest)
write.csv(imp, "imp.csv")

In [24]:
cor(imp, 1 - noisy_features)

,V1
imp_org,-0.1397322
imp_ranger,0.2618766
imp_cforest,0.1636625


In [36]:
library(pROC)
roc_obj <- roc(array(noisy_features), array(noisy_features))
auc(roc_obj)

ERROR: Error in roc.default(array(noisy_features), array(noisy_features)): 'response' must have two levels


In [ ]:
pos.scores <- imp$imp_org[noisy_features == 0]
neg.scores <- imp$imp_org[noisy_features == 1]
mean(sample(pos.scores,100000,replace=T) > sample(neg.scores,100000,replace=T))